In [1]:
#%%capture
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOptions = [2,2,5,9,10,16,15]
keffs = [1.0,1.0,1.0,0.99,0.98,0.995,0.953]
centralities = ["0%-93%"]
Opt_names = ["no ML eID","ML eID", "conv partner","tighter match","conv pattern", "ML conv rej", "ML hit, ddphi"]

In [3]:
N_centr = 6
N_options = 6
N_pt = 1
colors0=[1,2,root.kGreen+2,4,root.kCyan,root.kMagenta,root.kOrange+4,root.kGray,root.kGray,root.kBlack,root.kYellow,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
colors=[2,4,4,root.kGreen+2,root.kOrange+4,root.kMagenta,root.kGray,root.kGray,root.kBlack,root.kCyan,root.kOrange,root.kViolet,root.kCyan,root.kGreen,root.kOrange]
central_bins = [0,20,60,80,20,40,40,60,60,80,80,100]
pt_boarders = [0.4,0.7,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="input/data/"
file_names=["m_ee_DCA_531.root","m_ee_DCA_531.root","m_ee_DCA_532.root","m_ee_DCA_533.root","m_ee_DCA_534.root","m_ee_DCA_537.root"]#,"m_ee_DCA_535.root"],"m_ee_DCA_536.root"]
hist_names = [["inv_mass_stat_00","inv_mass_syst_00",f"inv_mass_ee_DCA_V{iOption}_FG12",f"inv_mass_ee_DCA_V{iOption}_BG12"] for iOption in iOptions]
N_type = len(hist_names[0])

In [4]:
hists_read = []

for iFile in range(len(file_names)):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hists_emc_read0.append(infile.Get(hist_names[iFile][itype]))
        try: hists_emc_read0[-1].SetDirectory(root.nullptr); hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"_{iFile}")
        except: 
            print(f"Error setting directory for {hist_names[iFile][itype]}{0} in file {file_names[iFile]}")
            break
        hists_emc_read0[-1].SetName(hists_emc_read0[-1].GetName()+f"{iFile}")
    hists_read.append(hists_emc_read0)

    infile.Close()


In [5]:
c0 = root.TCanvas(f"c0",f"c0",900,900)
legends =[]
hist0_stat = hists_read[0][0].Clone(); hist0_syst = hists_read[0][1].Clone()
hist1_stat = hists_read[1][0].Clone(); hist1_syst = hists_read[1][1].Clone()
hist2_stat = hists_read[2][0].Clone(); hist2_syst = hists_read[2][1].Clone()


Format_Hist_total(hist0_stat,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
hist0_stat.SetMarkerColorAlpha(colors0[0],(10-1)/10.); hist0_stat.SetLineColorAlpha  (colors0[0],(10-1)/10.)
hist0_syst.SetFillColorAlpha(colors0[0],0.4)

hist1_stat.SetMarkerColorAlpha(colors0[1],(10-1)/10.); hist1_stat.SetLineColorAlpha  (colors0[1],(10-1)/10.)
hist1_syst.SetFillColorAlpha(colors0[1],0.4)

hist2_stat.SetMarkerColorAlpha(colors0[2],(10-1)/10.); hist2_stat.SetLineColorAlpha  (colors0[2],(10-1)/10.)
hist2_syst.SetFillColorAlpha(colors0[2],0.4)

hist0_stat.Draw(""); hist0_syst.Draw("same e2")
hist2_stat.Draw("same"); hist2_syst.Draw("same e2")
hist1_stat.Draw("same"); hist1_syst.Draw("same e2")

root.gPad.SetLogy()
c0.Draw()


In [6]:
edges = array('d', [
    0.,0.025,0.05,0.075,0.1,0.15,0.25,0.35,0.45,0.65,0.7,0.75,0.8,0.9,0.95,1.0,1.05,
    1.15,1.3,1.5,1.75,2.0,2.25,2.5,2.75,2.9,3.0,3.05,3.1,3.15,3.2,3.3,3.45,
    3.55,3.65,3.75,3.85,4.5
])

nbins = len(edges) - 1

In [7]:
c1 = root.TCanvas(f"c1",f"c1",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = False
alphas = [0.5,0.3,0.3,0.3,0.3,0.3,0.3]
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        keff  = keffs[ifile]
        if if_divide: 
            if proj_BG.GetBinContent(ibin) >0:
                proj.SetBinContent(ibin,(proj.GetBinContent(ibin)/proj_BG.GetBinContent(ibin))/keff)
                proj.SetBinError(ibin,(proj.GetBinContent(ibin)**0.5/proj_BG.GetBinContent(ibin))/keff)
            else : 
                proj.SetBinContent(ibin,1)
        else:
            proj.SetBinContent(ibin,proj.GetBinContent(ibin) - keff*proj_BG.GetBinContent(ibin))
        if not if_divide: 
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
        proj_syst.SetBinError(ibin,((proj_BG.GetBinContent(ibin)/100./proj.GetBinWidth(ibin))**2+0*(proj_BG.GetBinError(ibin)/proj.GetBinWidth(ibin))**2)**0.5)
        if if_divide: 
            proj.SetBinError(ibin,proj.GetBinError(ibin)/(proj_BG.GetBinContent(ibin)+1))
    
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [8]:
c1 = root.TCanvas(f"c1",f"c1",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = False
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range (1,1+proj.GetNbinsX()):
        proj.SetBinError(ibin,(proj.GetBinContent(ibin))**0.5)
        keff  = 0
        if if_divide: 
            if proj_BG.GetBinContent(ibin) >0:
                proj.SetBinContent(ibin,(proj.GetBinContent(ibin)/proj_BG.GetBinContent(ibin))/keff)
                proj.SetBinError(ibin,(proj.GetBinContent(ibin)**0.5/proj_BG.GetBinContent(ibin))/keff)
            else : 
                proj.SetBinContent(ibin,1)
        else:
            proj.SetBinContent(ibin,proj.GetBinContent(ibin) - keff*proj_BG.GetBinContent(ibin))
        if not if_divide: 
            proj.SetBinContent(ibin,proj.GetBinContent(ibin)/proj.GetBinWidth(ibin))
            proj_syst.SetBinContent(ibin,proj.GetBinContent(ibin))
        proj.SetBinError(ibin,proj.GetBinError(ibin)/proj.GetBinWidth(ibin))
        proj_BG.SetBinError(ibin,abs(proj_BG.GetBinContent(ibin))**0.5)
        proj_syst.SetBinError(ibin,((proj_BG.GetBinContent(ibin)/100./proj.GetBinWidth(ibin))**2+0*(proj_BG.GetBinError(ibin)/proj.GetBinWidth(ibin))**2)**0.5)
        if if_divide: 
            proj.SetBinError(ibin,proj.GetBinError(ibin)/(proj_BG.GetBinContent(ibin)+1))
    
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [9]:
c1 = root.TCanvas(f"c2",f"c2",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = True
legend = root.TLegend(0.2,0.63,0.7,0.95); legends.append(legend)
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin)) / (keff * proj_BG.GetBinContent(ibin) +1e-48)-1)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{ee}, GeV","SG/BG, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Scale(0.2); proj_syst.Scale(0.2)
    if ifile == 0:  proj.Draw("")   
    proj.SetMaximum(3000)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [10]:
c1 = root.TCanvas(f"c2",f"c2",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = True
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX2_{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_sys2_t{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX2_{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        error = 0
        if proj.GetBinContent(ibin) > 0:
            error = proj.GetBinError(ibin) / (proj.GetBinContent(ibin) - keff * proj_BG.GetBinContent(ibin)) 
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin) - keff * proj_BG.GetBinContent(ibin)) / proj.GetBinContent(ibin)**0.5 )
            error *= abs(proj.GetBinContent(ibin)**0.5)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, error)
        #proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48)**0.5)
        #proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],alphas[ifile-ic])
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")
c1.Draw()

In [11]:
c1 = root.TCanvas(f"c2",f"c2",1400,2100)
c1.Divide(2,3)
legends =[]
if_divide = True
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin)) / (keff * proj_BG.GetBinContent(ibin) +1e-48)-1)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)

    for ibin in range(1, 1 + proj_syst.GetNbinsX()):
        proj_syst.SetBinContent(ibin, proj_syst.GetBinContent(ibin)/proj_syst.GetBinError(ibin))
        proj_syst.SetBinError(ibin, 1)
    
    Format_Hist_total(proj_syst,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
    proj_syst.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj_syst.SetLineColorAlpha  (colors0[ifile],(10-1)/10.)
    root.gPad.SetLogy(); 
    if ifile == 0:  proj_syst.Scale(0.2)
    if ifile == 0:  proj_syst.Draw("")   
    legend = root.TLegend(0.55,0.75,0.89,0.89); legends.append(legend)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj_syst,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        if ifile == 0 and ic !=0: proj_syst.Draw("")
        else: proj_syst.Draw("same")
c1.Draw()

In [12]:
c1 = root.TCanvas(f"c1",f"c1",900,900)
legends =[]
hist0_new_stat = hist0_stat.Rebin(1, hist0_stat.GetName()+"hist0_stat_rebinned"); hist0_new_syst = hist0_syst.Rebin(1, hist0_syst.GetName()+"hist0_syst_rebinned")
hist1_new_stat = hist1_stat.Rebin(1, hist1_stat.GetName()+"hist1_stat_rebinned"); hist1_new_syst = hist1_syst.Rebin(1, hist1_syst.GetName()+"hist1_syst_rebinned")
hist2_new_stat = hist2_stat.Rebin(1, hist2_stat.GetName()+"hist2_stat_rebinned"); hist2_new_syst = hist2_syst.Rebin(1, hist2_syst.GetName()+"hist2_syst_rebinned")  



for ibin in range(1,hist0_new_stat.GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    val0 = hist0_new_stat.GetBinContent(ibin)
    val1 = hist1_new_stat.GetBinContent(ibin)
    val2 = hist2_new_stat.GetBinContent(ibin)
    if val0>0:
        hist1_new_stat.SetBinContent(ibin, val1/val0)
        hist2_new_stat.SetBinContent(ibin, val2/val0)
        err1 = hist1_new_stat.GetBinError(ibin)
        err2 = hist2_new_stat.GetBinError(ibin)
        hist1_new_stat.SetBinError(ibin, err1/val0)
        hist2_new_stat.SetBinError(ibin, err2/val0)
        hist0_new_syst.SetBinContent(ibin, 1)
        hist0_new_syst.SetBinError(ibin, hist0_new_syst.GetBinError(ibin)/val0)
    else:
        hist1_new_stat.SetBinContent(ibin, 0)
        hist2_new_stat.SetBinContent(ibin, 0)
        hist1_new_stat.SetBinError(ibin, 0)
        hist2_new_stat.SetBinError(ibin, 0)

Format_Hist_total(hist1_new_stat,"m_{ee}, GeV","dN/dm_{ee}, GeV^{-1}",0.17,0.13,0.01,0.01,0.07,0.06,21,1.5,4,3,4,0.85,1.24)
hist0_new_stat.SetMarkerColorAlpha(colors0[0],(10-1)/10.); hist0_new_stat.SetLineColorAlpha  (colors0[0],(10-1)/10.)
hist0_new_syst.SetFillColorAlpha(colors0[0],0.3)

hist1_new_stat.SetMarkerColorAlpha(colors0[1],(10-1)/10.); hist1_new_stat.SetLineColorAlpha  (colors0[1],(10-1)/10.)
hist1_new_syst.SetFillColorAlpha(colors0[1],0.2)

hist2_new_stat.SetMarkerColorAlpha(colors0[2],(10-1)/10.); hist2_new_stat.SetLineColorAlpha  (colors0[2],(10-1)/10.)
hist2_new_syst.SetFillColorAlpha(colors0[2],0.2)

hist1_new_syst.Divide(hist0_new_syst); hist2_new_syst.Divide(hist0_new_syst)
#root.gPad.SetLogy(1)
hist1_new_stat.SetMaximum(2); hist1_new_stat.SetMinimum(0.1)
hist1_new_stat.Draw(""); #hist1_new_syst.Draw("same e2")
hist2_new_stat.Draw("same"); #hist2_new_syst.Draw("same e2")
hist0_new_syst.Draw("same e2")
#drawing dashed line at y=1
line = root.TLine(hist1_new_stat.GetXaxis().GetXmin(), 1, hist1_new_stat.GetXaxis().GetXmax(), 1)
line.SetLineStyle(2); line.SetLineWidth(4); line.SetLineColor(root.kBlack)
line.Draw("same")

c1.Draw()


In [13]:
c1 = root.TCanvas(f"c2",f"c2",1000,1000)
legends =[]
if_divide = True
legend = root.TLegend(0.2,0.58,0.7,0.9); legends.append(legend)
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX1{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_1syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_po1rjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin) - keff * proj_BG.GetBinContent(ibin) ) / (keff * proj_BG.GetBinContent(ibin) + proj.GetBinContent(ibin) +  1e-48)**0.5)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{e^{+}e^{-}} [GeV]","Signal/#sqrt{SG+BG}",0.17,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.85,1.34)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.); proj_syst.SetMarkerColorAlpha(0,0)
    root.gPad.SetLogy(); 
    if ifile == 0:  
        rand0 = root.gRandom.Rndm() * 5.0/20.0 + 0.1
        for ibin in range(1, 1 + proj.GetNbinsX()): proj.SetBinContent(ibin, proj.GetBinContent(ibin)*rand0)
        for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinContent(ibin, proj_syst.GetBinContent(ibin)*rand0)
    if ifile == 0:  proj.Draw("")   
    proj.SetMaximum(3000)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],0.1)
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")

legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[0]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
Format_Legend(legends[-1],0.05,12)
legends[-1].Draw()
c1.Draw()
#c1.SaveAs("output/mass/SG_BG_comparison.pdf")

In [14]:
c1 = root.TCanvas(f"c2",f"c2",1000,1000)
legends =[]
if_divide = True
legend = root.TLegend(0.2,0.58,0.7,0.9); legends.append(legend)
for ifile in range(len(file_names)):
    c1.cd(1+ifile)
    proj = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj_syst = hists_read[ifile][2].ProjectionX(hists_read[ifile][2].GetName()+f"_syst{ifile}",1,40,1,12)
    proj_BG = hists_read[ifile][3].ProjectionX(hists_read[ifile][3].GetName()+f"_porjX{ifile}",1,40,1,12)
    proj    = proj.Rebin(nbins, proj.GetName()+"_reb",    edges)
    proj_syst = proj_syst.Rebin(nbins, proj_syst.GetName()+"_reb", edges)
    proj_BG = proj_BG.Rebin(nbins, proj_BG.GetName()+"_reb", edges)


    for ibin in range(1, 1 + proj.GetNbinsX()):
        proj.SetBinError(ibin, (proj.GetBinContent(ibin)) ** 0.5)
        keff = keffs[ifile]
        if proj.GetBinContent(ibin) > 0:
            proj.SetBinContent(ibin, (proj.GetBinContent(ibin)) / (keff * proj_BG.GetBinContent(ibin) +1e-48)-1)
        else:
            proj.SetBinContent(ibin, 0)
        proj.SetBinError(ibin, proj.GetBinError(ibin) / (proj_BG.GetBinContent(ibin) + 1e-48))
        proj.SetBinError(ibin, ( proj.GetBinError(ibin)**2 )**0.5 )
    # Rebin the histogram with a factor of 2 starting from the value 3
    bin_start = proj.FindBin(3.6)
    bin_edges = [proj.GetBinLowEdge(i) for i in range(1, bin_start)] + \
                [proj.GetBinLowEdge(i) for i in range(bin_start, proj.GetNbinsX() + 1, 2)] + \
                [proj.GetBinLowEdge(proj.GetNbinsX() + 1)]
    proj_rebinned = proj.Rebin(len(bin_edges) - 1, f"reb_{proj.GetName()}", array('d', bin_edges))
    for ibin in range(bin_start, 1 + proj_rebinned.GetNbinsX()): proj_rebinned.SetBinContent(ibin, proj.GetBinContent(ibin)/2)
    proj_syst = proj_rebinned.Rebin(1,f"syst_{proj_rebinned.GetName()}")
    for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinError(ibin, 0.01)
    
    Format_Hist_total(proj,"m_{e^{+}e^{-}} [GeV]","Signal/Background",0.17,0.13,0.01,0.01,0.06,0.05,21,1.5,4,3,4,0.85,1.34)
    proj.SetMarkerColorAlpha(colors0[ifile],(10-1)/10.); proj.SetLineColorAlpha  (colors0[ifile],(10-1)/10.); proj_syst.SetMarkerColorAlpha(0,0)
    root.gPad.SetLogy(); 
    if ifile == 0:  
        rand0 = root.gRandom.Rndm() * 5.0/20.0 + 0.1
        for ibin in range(1, 1 + proj.GetNbinsX()): proj.SetBinContent(ibin, proj.GetBinContent(ibin)*rand0)
        for ibin in range(1, 1 + proj_syst.GetNbinsX()): proj_syst.SetBinContent(ibin, proj_syst.GetBinContent(ibin)*rand0)
    if ifile == 0:  proj.Draw("")   
    proj.SetMaximum(3000)
    legends[-1].SetBorderSize(0); legends[-1].SetFillStyle(0)
    legends[-1].AddEntry(proj,Opt_names[ifile],"lep")
    legends[-1].Draw()
    for ic in range(ifile,len(file_names)):
        c1.cd(1+ic)
        proj_syst.SetFillColorAlpha(colors0[ifile],0.1)
        if ifile == 0 and ic !=0: proj.Draw("")
        else: proj.Draw("same")
        proj_syst.Draw("same e2")

legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[0]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
Format_Legend(legends[-1],0.05,12)
legends[-1].Draw()
c1.Draw()
#c1.SaveAs("output/mass/SG_BG_comparison.pdf")

In [15]:
infile = root.TFile.Open("input/Star.root", "READ")

d = infile.Get("Table 1")                  # TDirectoryFile
hists_star = d.Get("Graph1D_y1")           # TGraphAsymmErrors

# Optional safety check
if not hists_star:
    raise RuntimeError("Graph not found")

hists_star.SetName("STAR_AuAu")             # optional
hists_star.SetTitle("STAR Au+Au 0–80%")

infile.Close()

In [16]:

# ---- extract x values once ----
n = hists_star.GetN()
x = hists_star.GetX()
#  ---- set x errors from neighbor spacing ----
for i in range(n):
    if i == 0:
        dx = 0.5 * (x[1] - x[0])
    elif i == n - 1:
        dx = 0.5 * (x[n-1] - x[n-2])
    else:
        dx = 0.5 * (x[i+1] - x[i-1])

    hists_star.SetPointEXlow(i,  dx/2)
    hists_star.SetPointEXhigh(i, dx/2)


In [17]:
accepance_eff_low_mass = root.TF1("accepance_eff_low_mass","pol1",0,0.7)
accepance_eff_low_mass.SetParameters(1.28898e-05,5.861e-06)
accepance_eff_high_mass = root.TF1("accepance_eff_high_mass","pol1",0.7,5.0 )
accepance_eff_high_mass.SetParameters(-5.34624e-05,0.000101395)

for ibin in range(1,hists_read[len(file_names)-1][0].GetNbinsX()+1):#dividing hist1 adh hist2  by hist0
    mass = hists_read[len(file_names)-1][0].GetBinCenter(ibin)
    if mass < 0.7:
        eff = accepance_eff_low_mass.Eval(mass)
    elif mass < 2.9:
        eff = accepance_eff_high_mass.Eval(mass)
    else: eff = accepance_eff_high_mass.Eval(2.0)
    if eff > 0:
        hists_read[len(file_names)-1][0].SetBinContent(ibin, hists_read[len(file_names)-1][0].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][0].SetBinError(ibin, hists_read[len(file_names)-1][0].GetBinError(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinContent(ibin, hists_read[len(file_names)-1][1].GetBinContent(ibin)/eff)
        hists_read[len(file_names)-1][1].SetBinError(ibin, hists_read[len(file_names)-1][1].GetBinError(ibin)/eff)
    

In [18]:
dnch_phenix = 550
dnch_star  = 180
iOpt = len(file_names)-1
hists_star.Scale(1./dnch_star)
hists_read[iOpt][0].Scale(1./dnch_phenix)
hists_read[iOpt][1].Scale(1./dnch_phenix)

In [19]:
canvases = []
legends =[]
for ipar in [0]:#[0,1,2,3,4,5]:
    c0 = root.TCanvas(f"c0_{ipar}",f"c0_{ipar}",1000,1000)
    c0.Divide(1,1)
    canvases.append(c0)
    iOpt = len(file_names)-1
    rebin = 1

    centr_length = [1,0.2/0.93,0.2/0.93,0.2/0.93,0.2/0.93,0.13/0.93]
    for icentr in range(ipar,ipar+1):
        legends.append(root.TLegend(0.2,0.89,0.99,0.97,""+centralities[icentr]+" Au+Au, #sqrt{s_{NN}} = 200 GeV"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()
        c0.cd(1)
        root.gPad.SetLogy()
        Format_Hist_total(hists_read[iOpt][0],"m_{e^{+}e^{-}} [GeV]","dN/dm_{e^{+}e^{-}}#divide dN_{ch}/d#eta|_{#eta=0} [GeV^{-1}]",0.175,0.13,0.01,0.01,0.06,0.05,21,1.5,1,3,1,0.99,1.35)
        hists_read[iOpt][0].Draw("")
        hists_read[iOpt][1].SetFillColorAlpha(1,0.4)
        hists_read[iOpt][1].Draw("same e2")
        legends.append(root.TLegend(0.25,0.83,0.99,0.89,"p_{T}^{e }>0.3 GeV, |y^{e}|<0.35"))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].Draw()

        legends.append(root.TLegend(0.25,0.63,0.8,0.92))
        Format_Legend(legends[-1],0.05,12)
        legends[-1].AddEntry(0,"","")
        legends[-1].AddEntry(hists_read[iOpt][0],"PHENIX","pl")
        legends[-1].AddEntry(hists_star,"#splitline{STAR #sqrt{s_{NN}} = 27 GeV}{0-80% Au+Au}","pl")
        #legends[-1].AddEntry(hists_star,"STAR #sqrt{s_{NN}} = 27 GeV 0-80% Au+Au","pl")
        legends[-1].Draw()
        


        #sum_sim_proj.Draw("same L")
        hists_read[iOpt][0].SetMaximum(2e-1)
        hists_read[iOpt][0].SetMinimum(5e-9)

        Format_Graph(hists_star,Msize=2,Mstyle=29,Lcolor=2,Mcolor=2,Lwidth=2)
        hists_star.Draw("P same")

    #if True: root.PHENIXTools.DrawPreliminary(0.6, 0.6, 0.3)


    # Draw the logo on the canvas with specified coordinates and size
    #logo.DrawImage(0.1, 0.8, 0.3, 0.9)  # Adjust the coordinates and size as needed

    c0.Draw()
    c0.Print(f"output/mass/mee_star.pdf")